# Imports

In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import FuncFormatter, PercentFormatter

plt.style.use('ggplot')

COLOR_FEMALE = cm.get_cmap("RdBu")(220) # set blue for female
COLOR_MALE = cm.get_cmap("RdBu")(35) # set red for male
TITLE_SIZE = 22
TITLE_PADDING = 10

import seaborn as sns

params = {
    'text.color': (0.25, 0.25, 0.25),
    'figure.figsize': [12, 5],
   }
plt.rcParams.update(params)

import pandas as pd
pd.options.display.max_rows = 500

import numpy as np
from numpy import percentile
np.random.seed(42)

import copy
import os
import sys
import glob
from shutil import copyfile
import time
from tabulate import tabulate
from tqdm.notebook import tqdm

import IPython.display as ipd

In [2]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, QuantileTransformer, PowerTransformer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, cross_val_predict
from sklearn.model_selection import GridSearchCV

from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process.kernels import RBF
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

from sklearn import set_config
set_config(display='diagram')

import lightgbm as lgb

import umap 

In [3]:
import librosa
import librosa.display

import pydub
from pydub import AudioSegment, effects

from aubio import source, pitch

# Helper functions

In [4]:
def sample_contributors(data, max_samples_per_contributor=100):
    # I set max_samples_per_contributor=100 because this is the average contribution 
    # per male client_id (vs just 14 for women).
    # I aim for a filtered data set which is as varied as possible and 
    # I try minimize the skew towards heavy contributors.
    g = data.groupby("client_id")
    tmp = []
    for _, data in g:
        if len(data) > max_samples_per_contributor:
            tmp.append(data.sample(max_samples_per_contributor, random_state=42))
        else:
            tmp.append(data)
    tmp = pd.concat(tmp)
#     tmp = tmp.reset_index(drop=True)
    return tmp
    
    
def prepare_metadata(df_, language):
    # drop all samples where label for target variable is missing
    df_.dropna(subset=["gender"], inplace=True)

    # simplify client_id
    mapping = dict(zip(df_.client_id.unique(), range(df_.client_id.nunique())))
    df_.client_id = df_.client_id.map(mapping)

    # map textual values for age to integers
    ages = ['teens', 'twenties', 'thirties', 'fourties', 'fifties', 'sixties',
           'seventies', 'eighties', 'nineties']
    ages_int = range(10, 100, 10)
    mapping = dict(zip(ages, ages_int))
    df_.age = df_.age.map(mapping)

    # get word_count as rough measure of recording length
    df_["word_count"] = df_.sentence.apply(lambda x: len(str(x).split()))

    # drop features that that I do not need
    df_.drop(["sentence", "segment", "locale", "up_votes", "down_votes"], axis=1, inplace=True)

    # drop gender «other» because ambiguous what that entails
    to_drop = df_[df_.gender=="other"].index
    df_.drop(to_drop, inplace=True)
    
    # remove samples with three words or less
    to_drop = df_[df_.word_count<=3].index
    df_.drop(to_drop, inplace=True)

    # drop recordings from teens because gender in that age is hard to distinguish
    to_drop = df_[df_.age==10].index
    df_.drop(to_drop, inplace=True)
    
    df_.reset_index(drop=True, inplace=True)
    
    # split male and female
    female = df_[df_.gender=="female"].copy()
    male = df_[df_.gender=="male"].copy()
    
    # reduce female samples to a max count of 100 per contributor to reduce bias towards heavy contributors
    female = sample_contributors(female, max_samples_per_contributor=100)
    male = sample_contributors(male, max_samples_per_contributor=100)

    # sample from men as many samples as I have female samples
    # again make sure that I include all voices and reduce bias towards men that have contributed heavily
    sampled_ids = []
    while len(sampled_ids)<len(female):
        idx = male.groupby("client_id").sample(1).index
        sampled_ids.extend(idx)
        male.drop(idx, inplace=True)
    male = df_.iloc[sampled_ids].copy()
    # reduce male samples to exact the sample size that we have for females
    male = male.sample(len(female), random_state=42)
    df_ = pd.concat([male, female])
        
    # add boolean for gender
    df_["man"] = np.where(df_.gender=="male", True, False)
    
    # reduce to essential columns
    cols = ["client_id", "path", "gender", "man"]
    df_ = df_[cols]

    # change path to filename (which it rather is) and add actual filepath according to language
    df_.rename({"path":"file_name"}, axis=1, inplace=True)
    base_path = f"_data/CommonVoice/{language}/clips/"
    df_["file_path"] = base_path + df_.file_name
    
    # reorder for aestethics ;0)
    cols = ['client_id', 'gender', 'man', 'file_name', 'file_path']
    df_ = df_[cols]
    
    df_.reset_index(drop=True, inplace=True)
    
    return df_

In [5]:
SAMPLE_RATE        = 48_000
MFCC_MIN_FREQUENCY = 60
MFCC_MAX_FREQUENCY = 8_000
MFCC_BANDS         = 80
FO_CUTOFF_UPPER    = 1_500
HOP_LENGTH         = 512


def get_fO(audio, sample_rate):
    # create empty array to return in case processing fails
    empty_ = np.empty(5,)
    empty_[:] = np.nan

    splits_cnt = int(len(audio)/HOP_LENGTH)
    splits = [audio[x * HOP_LENGTH:x * HOP_LENGTH + HOP_LENGTH] for x in range(0, splits_cnt)]
    
    # "yinfast" and "yin" both work fine and deliver almost identical results, yinfast being around 2x faster 
    # "yinfft" yields many NaNs
    pitch_o = pitch("yinfast", samplerate=sample_rate)
    pitches = []
    for split in splits:
        pitch_ = pitch_o(split)[0]
        confidence = pitch_o.get_confidence()
        if confidence < 0.8: 
            pitch_ = 0.
        pitches.append(pitch_) 

    if len(pitches)==0:
        return empty_

    fO = [0.0 if x>FO_CUTOFF_UPPER else x for x in pitches]
    fO = [x for x in fO if x!=0.0]

    if len(fO)==0:
        return empty_ 

    return (np.mean(fO), np.median(fO), np.std(fO), np.min(fO), np.max(fO))


def pydub_to_np(audio):
    return np.array(audio.get_array_of_samples(), dtype=np.float32).reshape((-1, audio.channels)).T


def detect_leading_silence(audio, silence_threshold=-50.0, chunk_size=10):
    trim_ms = 0 # ms
    assert chunk_size > 0 # to avoid infinite loop
    while audio[trim_ms:trim_ms+chunk_size].dBFS < silence_threshold and trim_ms < len(audio):
        trim_ms += chunk_size
    return trim_ms


def get_features(file, secs_to_process=2):
    # create empty array to return in case processing fails
    empty_ = np.empty(85,)
    empty_[:] = np.nan 
    
    audio_file = AudioSegment.from_file(file)
    sample_rate = audio_file.frame_rate

    # normalize audio levels
    try:
        audio_file = effects.normalize(audio_file)  
    except Exception as e:
        print(f"{file} can't be normalized")
        return empty_  
    
    # remove silence at beginning and end
    start_trim = detect_leading_silence(audio_file)
    end_trim = detect_leading_silence(audio_file.reverse())
    audio_duration = len(audio_file) 
    audio_file = audio_file[start_trim:audio_duration-end_trim]
    
    if audio_file.duration_seconds > secs_to_process:
        ms_to_process = secs_to_process*1000
        center_of_audio = len(audio_file)/2 
        start_extract = center_of_audio - ms_to_process/2 
        end_extract = start_extract + ms_to_process
        audio_file = audio_file[start_extract:end_extract]
        
    waveform = pydub_to_np(audio_file)

    if waveform.ndim == 2: # check if file is stereo 
        waveform = waveform.mean(axis=0) # if stereo merge channels by averaging

    if len(waveform)>0:
        fO_ = get_fO(waveform, sample_rate)
        mfcc_ = np.mean(librosa.feature.mfcc(y=waveform, 
                                             sr=sample_rate, 
                                             n_mfcc=MFCC_BANDS, 
                                             fmin=MFCC_MIN_FREQUENCY,
                                             fmax=MFCC_MAX_FREQUENCY,
                                            ), axis=1) 
        return np.hstack([fO_, mfcc_])
    
    else:
        return empty_    

    
def get_cols():
    mel_to_Hz = librosa.mel_frequencies(n_mels=MFCC_BANDS, 
                                        fmin=MFCC_MIN_FREQUENCY, 
                                        fmax=MFCC_MAX_FREQUENCY)
    
    cols = ["fO_mean", "fO_median", "fO_std", "fO_min", "fO_max"]
    cols_mel = [f"{x:.0f}" + "_Hz" for x in mel_to_Hz]
    cols.extend(cols_mel)
    return cols
    
    
def process_audio_files(file_paths, secs_to_process=2):
    features = []
    for file_path in tqdm(file_paths):
        features.append(get_features(file_path, secs_to_process=secs_to_process))
    features = pd.DataFrame(features)
    features.columns = get_cols()
    return features

# 1) Creating improved features
- In the previous notebooks I **retrieved the FO from the full audio file** but **extracted the MFCC from the middle segment of the file** with n seconds length. I **improve the validity of the extracted features by calculating the FO from the exact same audio segment as the MFCC.** This will degrade model accuracy somewhat since the FO is calculated based on much less information. At the same time this procedure is **more representative in regard to the podcast analysis** where I will cut episodes in segments of n seconds length. 
- I improved the extraction speed by loading the file just once and calculating FO and MFCC in one go.
- I also changed the aubio algorithm for FO pitch extraction from `yin` to `yinfast` that yields almost identical results while being at least 2x faster.

## Extracting improved features for CommonVoice
Creating improved features for all languages.

In [6]:
# %%time
# languages = ["_german", "_french", "_italian", "_romansh"]

# for language in languages:
#     tmp = pd.read_csv(f"_data/CommonVoice/{language}/validated.tsv", sep="\t")
#     meta = prepare_metadata(tmp, language)
#     feat = process_audio_files(meta.file_path)
    
#     # dropping recordings for which FO or MFCC couldn't be computed 
#     feat.dropna(inplace=True)
#     meta = meta.loc[feat.index]

#     meta.reset_index(drop=True, inplace=True)
#     feat.reset_index(drop=True, inplace=True)
    
#     meta.to_parquet(f"_saved_features/cv{language}_meta.parq")
#     feat.to_parquet(f"_saved_features/cv{language}_features.parq")

In [7]:
# # load single language
# language = "_german"
# meta = pd.read_parquet(f"_saved_features/cv{language}_meta.parq")
# feat = pd.read_parquet(f"_saved_features/cv{language}_features.parq")

From experiments I found that **I get slightly better results if I train on all languages used in Switzerland.**

In [8]:
# load features for multiple languages
metas = []
feats = []

languages = ["_german", "_french", "_italian", "_romansh"]
for language in languages:
    metas.append(pd.read_parquet(f"_saved_features/cv{language}_meta.parq"))
    feats.append(pd.read_parquet(f"_saved_features/cv{language}_features.parq"))

meta = pd.concat(metas).reset_index(drop=True)
feat = pd.concat(feats).reset_index(drop=True)

# 2) Creating an additional test data set from podcasts
- **Until now I only examined and trained on CommonVoice data.** This was practical in order to analyze gender prediction. At the same time **the CommonVoice data is not fully representative in regards to the target domain of podcast media.**
- To more thoroughly test the accuracy of the trained models I **created an additional data set derived from actual podcasts episodes and with  Swissgerman samples.**
- I used unsupervised learning with KMeans to quickly cluster voice samples from various podcast episodes. I checked these samples manually and fixed erroneous samples. 
- **The podcast test set contains 2'200 audio samples** with 1'400 recordings in German, 650 in Swissgerman, 100 Italian and 50 English ones.

In [9]:
podcast_samples = glob.glob("_data/_podcasts/podcasts_testdata/*/*")
meta_pod = pd.DataFrame([x.split("/")[3:] for x in podcast_samples], columns=["gender", "file_name"])
meta_pod["file_path"] = "_data/_podcasts/podcasts_testdata/" + meta_pod.gender + "/" + meta_pod.file_name
meta_pod["language"] = meta_pod.file_name.apply(lambda x: x.split("_")[0])
cols = ['gender', 'language', 'file_name', 'file_path']
meta_pod = meta_pod[cols]

print(tabulate(meta_pod.gender.value_counts().to_frame()))
print()
print("Languages in podcast test set:")
print(tabulate(meta_pod.language.value_counts().to_frame()))

------  ----
male    1100
female  1100
------  ----

Languages in podcast test set:
--  ----
de  1400
ch   650
it   100
en    50
--  ----


In [10]:
# retrieving audio features for podcast test set
feat_pod = process_audio_files(meta_pod.file_path)

meta_pod.to_parquet("_saved_features/meta_pod.parq")
feat_pod.to_parquet("_saved_features/feat_pod.parq")

  0%|          | 0/2200 [00:00<?, ?it/s]

In [11]:
meta_pod = pd.read_parquet("_saved_features/meta_pod.parq")
feat_pod = pd.read_parquet("_saved_features/feat_pod.parq")

print(tabulate(meta_pod.gender.value_counts().to_frame()))
print()
print("Languages in podcast test set:")
print(tabulate(meta_pod.language.value_counts().to_frame()))

------  ----
male    1100
female  1100
------  ----

Languages in podcast test set:
--  ----
de  1400
ch   650
it   100
en    50
--  ----


# 3) Training models on improved features
I do again a gridsearch with the most promising classifiers that I found in the previous step. 

In [12]:
X_train = feat
y_train = meta.gender
X_test = feat_pod
y_test = meta_pod.gender

In [13]:
results_grid = []
results_best = []

In [14]:
def gridsearch_classifier(params_grid):
    pipe = Pipeline(steps=[('scaler', StandardScaler()), 
                           ('estimator', LogisticRegression())])

    grid = GridSearchCV(pipe, params_grid, n_jobs=-1, verbose=3)
    grid.fit(X_train, y_train)
    accuracy_validated = grid.best_score_
    
    y_pred = grid.predict(X_test)
    accuracy_test_set = accuracy_score(y_test, y_pred)
    
    print("-"*80)
    print(f"{accuracy_validated:.3f} accuracy CV 5fold")
    print(f"{accuracy_test_set:.3f} accuracy on podcast test set\n")
    
    results_detailed = pd.DataFrame(grid.cv_results_)
    cols = list(params_grid[0].keys())
    cols = ["param_" + x for x in cols]
    cols = sorted(cols)
    cols.extend(['mean_test_score', 'std_test_score', 'rank_test_score', "mean_fit_time"])
    results_detailed = results_detailed.loc[:, cols]
    results_detailed.param_estimator = results_detailed.param_estimator.apply(lambda x: str(x).split("(")[0])
    results_detailed.columns = results_detailed.columns.str.replace("param_", "")

    display(results_detailed.sort_values("mean_test_score", ascending=False)
           .style
           .highlight_max(color="lightgreen", subset=["mean_test_score"])
           .highlight_min(color="lightgreen", subset=["std_test_score", "rank_test_score"])
           .background_gradient(cmap='Reds', subset=['mean_fit_time'])
           .hide_index()
           .set_precision(3)
           )
    print()
    print("Best parameters are")
    print(grid.best_params_)
    print("-"*80)
    print()
    
    return grid, results_detailed, accuracy_validated, accuracy_test_set

In [15]:
classifiers = [
    QuadraticDiscriminantAnalysis(),
    LogisticRegression(max_iter=1e3),
    KNeighborsClassifier(),
    SVC(kernel="rbf"),
    lgb.LGBMClassifier()
]

- **All 5 classifiers yield good results crossvalidated (tested on the CommonVoice samples) in a close range of 0.933 to 0.948 accuracy.**
- **SVC ranks best closely followed by LightGBM and Logistic regression.** 
- Unfortunately **SVC trains very slowly** (it'll be even slower when I train with `probability=True`).
- Since Logistic Regression trains very fast with good results I examine this classifier more closely.

In [16]:
%%time
params_grid = [{"scaler": [StandardScaler(), 
                           QuantileTransformer()],
                'estimator': classifiers,
               }]

_, _, _, _= gridsearch_classifier(params_grid)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
--------------------------------------------------------------------------------
0.948 accuracy CV 5fold
0.926 accuracy on podcast test set



estimator,scaler,mean_test_score,std_test_score,rank_test_score,mean_fit_time
SVC,StandardScaler(),0.948,0.008,1,364.563
SVC,QuantileTransformer(),0.947,0.007,2,251.699
LGBMClassifier,StandardScaler(),0.946,0.005,3,4.493
LGBMClassifier,QuantileTransformer(),0.945,0.005,4,5.266
LogisticRegression,StandardScaler(),0.943,0.005,5,0.734
LogisticRegression,QuantileTransformer(),0.943,0.005,6,2.815
KNeighborsClassifier,QuantileTransformer(),0.936,0.011,7,1.642
QuadraticDiscriminantAnalysis,QuantileTransformer(),0.935,0.006,8,2.416
KNeighborsClassifier,StandardScaler(),0.935,0.009,9,0.203
QuadraticDiscriminantAnalysis,StandardScaler(),0.933,0.004,10,1.386



Best parameters are
{'estimator': SVC(), 'scaler': StandardScaler()}
--------------------------------------------------------------------------------

CPU times: user 3min 47s, sys: 1.88 s, total: 3min 49s
Wall time: 13min 37s


In [17]:
%%time
params_grid = [{"scaler": [StandardScaler()],
                'estimator':[LogisticRegression(max_iter=1e3)],
                'estimator__C': np.logspace(-1, 1, 20),
                }
              ]

_, _, _, _ = gridsearch_classifier(params_grid)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
--------------------------------------------------------------------------------
0.944 accuracy CV 5fold
0.926 accuracy on podcast test set



estimator,estimator__C,scaler,mean_test_score,std_test_score,rank_test_score,mean_fit_time
LogisticRegression,0.336,StandardScaler(),0.944,0.005,1,0.837
LogisticRegression,0.162,StandardScaler(),0.944,0.005,2,0.800
LogisticRegression,0.207,StandardScaler(),0.944,0.005,3,0.854
LogisticRegression,0.264,StandardScaler(),0.943,0.005,4,0.834
LogisticRegression,0.546,StandardScaler(),0.943,0.005,5,0.823
LogisticRegression,1.833,StandardScaler(),0.943,0.005,6,0.845
LogisticRegression,0.428,StandardScaler(),0.943,0.005,7,0.822
LogisticRegression,10.000,StandardScaler(),0.943,0.005,8,0.689
LogisticRegression,0.695,StandardScaler(),0.943,0.005,9,0.835
LogisticRegression,7.848,StandardScaler(),0.943,0.005,10,0.822



Best parameters are
{'estimator': LogisticRegression(C=0.33598182862837817, max_iter=1000.0), 'estimator__C': 0.33598182862837817, 'scaler': StandardScaler()}
--------------------------------------------------------------------------------

CPU times: user 2.78 s, sys: 697 ms, total: 3.48 s
Wall time: 12.7 s


# 4) Evaluating Logistic regression trained with the improved features

Since Logistic regression trains fast and yields very good results I examine its predictions in greater detail.

In [18]:
def train_model(X_train, y_train, X_test, y_test, cv_folds=5,
                clf=LogisticRegression(max_iter=1e3, C=0.4),
                report=True):
    
    pipe = make_pipeline(QuantileTransformer(), clf)   
    
    scores = cross_val_score(pipe, X_train, y_train, cv=cv_folds, n_jobs=-1)
    print(f"{np.mean(scores):.3f} accuracy crossvalidated {cv_folds}fold")

    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    print(f"{accuracy_score(y_test, y_pred):.3f} accuracy on test set")
    print()
    
    if report == True:
        print(classification_report(y_test, y_pred, zero_division=False))

        cfmtrx = confusion_matrix(y_test, y_pred, normalize=None)
        display(pd.DataFrame(cfmtrx, 
                     index=[x + "_true" for x in ["female", "male"]],
                     columns=[x + "_pred" for x in ["female", "male"]]
                    ))
        
    return pipe

- Logistic regression yields close to 0.93 accuracy.
- Precision for male voices is a little less good (on this small test data set). This is due to many samples of two female voices being missclassified (see below).

In [19]:
pipe = train_model(X_train, y_train, 
                   X_test, y_test)

0.943 accuracy crossvalidated 5fold
0.926 accuracy on test set

              precision    recall  f1-score   support

      female       0.94      0.91      0.92      1100
        male       0.91      0.94      0.93      1100

    accuracy                           0.93      2200
   macro avg       0.93      0.93      0.93      2200
weighted avg       0.93      0.93      0.93      2200



,female_pred,male_pred
female_true,1000,100
male_true,63,1037


From **checking the wrong predictions of the classifier** I gather:
- **Many false predictions seem comprehensible.**
- **Almost all samples of two female speakers are wrongly predicted as male samples** (both have particularly deep voices). These errors alone account for around 4% of the 7% total errors.
- Several missclassified male speakers have particularly playful, energetic and expressive voices, that seem higher pitched too.
- Many of the Italian male samples are missclassified (again: very playful and high pitch vocal expression).

In [20]:
# tmp = meta_pod.copy()
# preds = pipe.predict(X_test)
# preds_bool = [True if x=="male" else False for x in preds]
# proba = pipe.predict_proba(X_test)
# tmp["predicted"] = preds
# tmp["predicted_proba"] = proba.max(axis=1)
# cols = ['language', 'file_name', 'gender', 'predicted']
# display(tmp[cols][tmp.gender!=tmp.predicted].sort_values("file_name")
#        .style
#        .hide_index())